# Predict Waist Circumference with Diffusion Weighted Imaging

This notebook using diffusion weighted imaging data, and subjects waist circumference in cm from the ABCD Study. 

In [2]:
import BPt as bp
import pandas as pd
import os

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
ps = bp.ProblemSpec(n_jobs=8)
ps.subjects

'all'

## Load the data needed

Data is loaded from a large csv file with all of the features from release 2 of the ABCD study.

In [ ]:
def load_from_rds(names, eventname='baseline_year_1_arm_1'):
    
    data = pd.read_csv('data/nda_rds_201.csv',
                       usecols=['src_subject_id', 'eventname'] + names,
                       na_values=['777', 999, '999', 777])
    
    data = data.loc[data[data['eventname'] == eventname].index]
    data = data.set_index('src_subject_id')
    data = data.drop('eventname', axis=1)
    
    # Obsificate subject ID for public example
    data.index = list(range(len(data)))
    
    # Return as pandas DataFrame cast to BPt Dataset
    return bp.Dataset(data)

In [ ]:
# This way we can look at all column avaliable
all_cols = list(pd.read_csv('data/nda_rds_201.csv', nrows=0))
all_cols[:10]

In [ ]:
# The target variable
target_cols = ['anthro_waist_cm']

# non input feature - i.e., those that inform 
non_input_cols = ['sex', 'rel_family_id']

# We will use the fiber at dti measures
dti_cols = [c for c in all_cols if '_fiber.at' in c and '.full.' in c]
len(dti_cols)

Now we can use the helper function defined at the start to load these features in as a Dataset

In [ ]:
data = load_from_rds(target_cols + non_input_cols + dti_cols)
data.shape

In [ ]:
# This is optional, but will print out some extra verbosity when using the dataset operations
data.verbose = 1

The first step we will do is tell the dataset what roles the different columns are. See: https://sahahn.github.io/BPt/user_guide/role.html

In [ ]:
data = data.set_target(target_cols) # Note we doing data = data.func()
data = data.set_non_input(non_input_cols)
data

A few things to note right off the bat.

1. The verbosity printed us out two statements, about dropping rows. This is due to a constraint on columns of role 'non input' that there cannot be any NaN / missing data, so those lines just say 2 NaN's were found when loading the first non input column and 6 when loading the next.

2. The values for sex are still 'F' and 'M', we will handle that next.

3. Some columns with role data are missing values. We will handle that as well.

In [ ]:
# We explicitly say this variable should be binary
data.to_binary('sex', inplace=True)

# We will ordinalize rel_family_id too
data = data.ordinalize(scope='rel_family_id')

data['non input']

Next let's look into that NaN problem we saw before.

In [ ]:
data.nan_info()

Seems like most of the missing data is missing for everyone, i.e., if the above info founds columns with only a few missing values, we might want to do something different, but this tells us that when data is missing it is missing for all columns.

We just drop any subjects with any NaN data below across the target variable and the Data

In [ ]:
data = data.drop_nan_subjects(scope='all')

Another thing we need to worry about with data like this is corrupted data, i.e., data with values that don't make sense due to a failure in the automatic processing pipeline. Let's look at the target variable first, then the data.

In [ ]:
data.plot('target')
data['target'].max(), data['target'].min(),

Yeah I don't know about that waist cm of 0 ...
The below code can be used to try different values of outliers to drop, since it is not by default applied in place.

In [ ]:
data.filter_outliers_by_std(scope='target', n_std=5).plot('target')

5 std seems okay, so let's actually apply it.

In [ ]:
data.filter_outliers_by_std(scope='target', n_std=5, inplace=True)

Let's look at the distribution of skew values for the dti data

In [ ]:
data['data'].skew().sort_values()

Looks okay, let's choose the variable with the most extreme skew to plot.

In [ ]:
data.plot(scope='dmri_dti.full.fa_fiber.at_fmaj')

How about we apply just a strict criteria of say 10 std.

In [ ]:
data.filter_outliers_by_std(scope='data', n_std=10, inplace=True)

## Define a Test set. 

In this example project we are going to test a bunch of different Machine Learning Pipeline's. In order to avoid meta-issues of overfitting onto our dataset, we will therefore define a train-test split. The train set we will use to try different pipelines, then only with the best final pipeline will we use the test set. 

We will impose one extra constraint when applying the test split, namely that members of the same family, i.e., those with the same family id, stay in the same training or testing fold.

In [ ]:
# We use this to say we want to preserve families
preserve_family = bp.CVStrategy(groups='rel_family_id')

# Apply the test split
data = data.set_test_split(size=.2,
                           cv_strategy=preserve_family,
                           random_state=6)

data

## Evaluate Different Pipelines

First let's save some commonly used parameters in an object called the ProblemSpec, we will use all defaults except for the number of jobs, for that let's use n_jobs=8.

In [ ]:
ps = bp.ProblemSpec(n_jobs=8)

The function we will use to evaluate different pipelines is bp.evaluate, let's start with an example with just a linear regression model.

In [ ]:
linear_model = bp.Model('linear')

results = bp.evaluate(pipeline=linear_model,
                      dataset=data,
                      problem_spec=ps,
                      eval_verbose=1)
results

Note from the verbose output that it has correctly detected a number of things including: a regression problem type, that we only want to use the training set and ... 

We get an instance of BPtEvaluator with the results. This stores all kinds of different information, for example:

In [ ]:
# Beta weights
results.get_fis()

In [ ]:
# Raw predictions made from each fold
results.get_preds_dfs()[0]

All options are listed under: 'Saved Attributes' and 'Avaliable Methods'.

Anyways, let's continue trying different models. We will use a ridge regression model. Let's also use the fact that the jupyter notebook is defining variables in global scope to clean up the evaluation code a bit so we don't have to keep copy and pasting it.

In [ ]:
def eval_pipe(pipeline, **kwargs):
    return bp.evaluate(pipeline=pipeline,
                       dataset=data,
                       problem_spec=ps,
                       verbose=1,
                       **kwargs)

In [ ]:
ridge_model = bp.Model('ridge')

# Add standard scaler before ridge model
# We want this because the same amount of regularization is used across features
ridge_pipe = bp.Pipeline([bp.Scaler('standard'), ridge_model])

eval_pipe(ridge_pipe)

The ridge regression has hyper-parameters though, what if just whatever the default value is, is not a good choice? We can add a parameter search to the model object.

In [ ]:
random_search = bp.ParamSearch('RandomSearch', n_iter=64)

ridge_search_model = bp.Model('ridge',
                              params=1, # Referring to a preset distribution of hyper-params
                              param_search=random_search)

ridge_search_pipe = bp.Pipeline([bp.Scaler('standard'), ridge_search_model])
    
eval_pipe(ridge_search_pipe)

At any point we can also ask different questions, for example: What happens we evaluate the model on only one sex?

In [ ]:
# Male only model first
eval_pipe(ridge_search_pipe,
          subjects=bp.ValueSubset('sex', 'M', decode_values=True))

In [ ]:
eval_pipe(ridge_search_pipe,
          subjects=bp.ValueSubset('sex', 'F', decode_values=True))

We do see a decrease in performance for the male model, though it is a bit difficult to tell if that is just noise, or related to the smaller sample sizes. Atleast the results are close, which tells us that sex likely is not being exploited as a proxy! For example if we ran the two same sex only models and they did terrible, it would tell us that our original model had been just memorizing sex effects.

Let's try a different choice of scaler next.

In [ ]:
ridge_search_pipe = bp.Pipeline([bp.Scaler('robust'), ridge_search_model])
eval_pipe(ridge_search_pipe)

In [ ]:
ridge_search_pipe = bp.Pipeline([bp.Scaler('quantile norm'), ridge_search_model])
eval_pipe(ridge_search_pipe)

Alright, thats a little bit of a boost, though notably the std between folds is higher. Let's keep it for now.

What about different choices of hyper-parameter optimization?

In [ ]:
# Let's just make the changes in place
ridge_search_pipe.steps[1].param_search.search_type = 'TwoPointsDE'
eval_pipe(ridge_search_pipe)

In [ ]:
ridge_search_pipe.steps[1].param_search.search_type = 'AdaptiveDiscreteOnePlusOne'
eval_pipe(ridge_search_pipe)

Okay so far we arn't really seeing sweeping differences when any of the parameters are changed, what about if we try some different complete models as well? Let's also start off by using some full default pipelines. We can see options with:

In [ ]:
from BPt.default.pipelines import pipelines
list(pipelines)

Let's try the elastic net first.

In [ ]:
from BPt.default.pipelines import elastic_pipe
elastic_pipe

We can see that this object has imputation and also a one hot enocer for categorical variables built in. In our case those will just be skipped.

In [ ]:
eval_pipe(elastic_pipe)

Let's try a light gbm model next

In [ ]:
from BPt.default.pipelines import lgbm_pipe
eval_pipe(lgbm_pipe)

non-linear svm?

In [ ]:
from BPt.default.pipelines import svm_pipe
eval_pipe(svm_pipe)

How about a linear svm?

In [ ]:
sgd_pipe = bp.Pipeline([bp.Scaler('robust'),
                        bp.Model('linear svm', params=1, param_search=random_search)])
eval_pipe(sgd_pipe)

## Compare

We can also notably achieve a lot of the different modelling steps we just took, but in a much cleaner way, that is through special Compare input objects. For example let's run the comparisons between a few models. Compare basically does a grid search over all of the parameters, but in contrast to setting up the different options as a nested grid search, the full 5-fold CV is run for every combination.

In [ ]:
# Define a set of bp.Options as wrapped in bp.Compare
compare_pipes = bp.Compare([bp.Option(sgd_pipe, name='sgd'),
                            bp.Option(ridge_search_pipe, name='ridge'),
                            bp.Option(elastic_pipe, name='elastic'),
                            bp.Option(lgbm_pipe, name='lgbm')])

# Pass as before as if a pipeline
results = eval_pipe(compare_pipes)
results.summary()